In [1]:
import pandas as pd

In [2]:
file = pd.read_csv('./all_csore.csv')

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_41168\3951011589.py:1: DtypeWarning: Columns (1,2,22) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('./all_csore.csv')


In [3]:
file

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,...,bucket_scoreacc_20250301_0830mnt,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,avg_digit1,project_code_name,total_digit1,final_score
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,...,13.0,13.0,13.0,9.0,8.0,7.0,10.5,NaN,H,HH07
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,NaN,H,HH07
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,L,LH07
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,NaN,H,HH07
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,NaN,H,HH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,D,DM06
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
